In [5]:
import cv2

# Load the video
cap = cv2.VideoCapture('./data/D19_20230511115642.mp4')

# Create a background subtractor object
back_sub = cv2.createBackgroundSubtractorMOG2()

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    back_sub_mask = back_sub.apply(frame)

    # Apply further preprocessing steps (e.g., erosion, dilation)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    back_sub_mask = cv2.morphologyEx(back_sub_mask, cv2.MORPH_OPEN, kernel)

    # Display the processed frame
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', back_sub_mask)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [4]:
#Resized
import cv2

# Load the video
cap = cv2.VideoCapture('./data/D19_20230511115642.mp4')

# Create a background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Initialize the first frame as the background
ret, frame = cap.read()
if not ret:
    print("Error: Failed to read the video file.")
    exit()

fgbg.apply(frame)  # Apply the first frame to initialize the background model

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgmask = fgbg.apply(frame)

    # Apply further preprocessing steps (e.g., erosion, dilation)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Resize the windows for better display
    frame = cv2.resize(frame, (640, 360))
    fgmask = cv2.resize(fgmask, (640, 360))

    # Display the processed frame and foreground mask
    cv2.imshow('Frame', frame)
    cv2.moveWindow('Frame', 100, 100)  # Move the window position
    cv2.imshow('Foreground Mask', fgmask)
    cv2.moveWindow('Foreground Mask', 800, 100)  # Move the window position

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [7]:
# Skipping first 60 frames and background sub

import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('./data/D19_20230511115642.mp4')

# Initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Initialize object detector and tracker
object_detector = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=64, detectShadows=True)
tracker = cv2.legacy.MultiTracker.create()

# Skip the first 60 frames for better background modeling
for _ in range(60):
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read the video file.")
        exit()
    fgbg.apply(frame, learningRate=0.001)

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgmask = fgbg.apply(frame, learningRate=0.001)

    # Apply object detection
    obj_mask = object_detector.apply(frame)
    _, obj_mask = cv2.threshold(obj_mask, 254, 255, cv2.THRESH_BINARY)
    obj_contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Update object tracker
    boxes = []
    for contour in obj_contours:
        area = cv2.contourArea(contour)
        if area > 500:  # Ignore small contours
            x, y, w, h = cv2.boundingRect(contour)
            boxes.append((x, y, w, h))
    tracker.update(frame, boxes)

    # Draw bounding boxes for tracked objects
    for box in tracker.getObjects():
        x, y, w, h = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Apply further preprocessing steps (e.g., erosion, dilation)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Colorize the foreground mask
    fgmask = cv2.applyColorMap(fgmask, cv2.COLORMAP_JET)

    # Resize the windows for better display
    frame = cv2.resize(frame, (640, 360))
    fgmask = cv2.resize(fgmask, (640, 360))

    # Display the processed frame and foreground mask
    cv2.imshow('Frame', frame)
    cv2.moveWindow('Frame', 100, 100)
    cv2.imshow('Foreground Mask', fgmask)
    cv2.moveWindow('Foreground Mask', 800, 100)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'update'
> Overload resolution failed:
>  - legacy_MultiTracker.update() takes at most 1 argument (2 given)
>  - legacy_MultiTracker.update() takes at most 1 argument (2 given)


In [10]:
import cv2

# Load the video
cap = cv2.VideoCapture('./data/D19_20230511115642.mp4')

# Initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Initialize object detector
object_detector = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=64, detectShadows=True)

# Initialize dictionary to store trackers
trackers = {}

# Skip the first 60 frames for better background modeling
for _ in range(60):
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read the video file.")
        exit()

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgmask = fgbg.apply(frame, learningRate=0.001)

    # Apply object detection
    obj_mask = object_detector.apply(frame)
    _, obj_mask = cv2.threshold(obj_mask, 254, 255, cv2.THRESH_BINARY)
    obj_contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Update object trackers
    for contour in obj_contours:
        area = cv2.contourArea(contour)
        if area > 500:  # Ignore small contours
            x, y, w, h = cv2.boundingRect(contour)
            # Create a new tracker if not already created
            if (x, y, w, h) not in trackers:
                trackers[(x, y, w, h)] = cv2.legacy.TrackerCSRT_create()
                trackers[(x, y, w, h)].init(frame, (x, y, w, h))
            else:
                # Update existing tracker
                trackers[(x, y, w, h)].update(frame)

            # Draw bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Apply further preprocessing steps (e.g., erosion, dilation)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Colorize the foreground mask
    fgmask = cv2.applyColorMap(fgmask, cv2.COLORMAP_JET)

    # Resize the windows for better display
    frame = cv2.resize(frame, (640, 360))
    fgmask = cv2.resize(fgmask, (640, 360))

    # Display the processed frame and foreground mask
    cv2.imshow('Frame', frame)
    cv2.moveWindow('Frame', 100, 100)
    cv2.imshow('Foreground Mask', fgmask)
    cv2.moveWindow('Foreground Mask', 800, 100)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
